In [1]:
!python --version

Python 3.10.12


In [ ]:
MODEL_NAME = 'Qwen/Qwen2.5-1.5B-Instruct-AWQ'
TASK = 'text-generation'

# ITER_START = 1
# ITER_TRIALS = 1

#SC_CHANGE
FILEDIR = "/kaggle/working/qwen2-1.5B/iter_0_sc/"
DATASET_FILEPATH = "/kaggle/input/squad-v2-processed/squad_2_with_few_shot_gpt2_5000.parquet"

HF_TOKEN = "HF_TOKEN" # input huggingface token

# Load Dependencies

In [3]:
from langchain_core.prompts import PromptTemplate
from tqdm import tqdm  # For progress bars

In [4]:
!pip install -qU transformers
!pip install -qU accelerate
!pip install -qU bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.1/512.1 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.8 MB/s eta 0:00:00


In [5]:
!pip install autoawq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.6/155.6 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 91.5 MB/s eta 0:00:00
  Created wheel for autoawq: filename=autoawq-0.2.9-py3-none-any.whl size=115106 sha256=af725b1610cdc0e6c7d7cbc5ee84f275ac141f0b7b18e2dc5640a5b9289f500d
  Stored in directory: /root/.cache/pip/wheels/13/2d/f6/3161d1c3acde652ce9b205d1c70eba821339e6d9a92dd81e12
Successfully built autoawq


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed, pipeline, BitsAndBytesConfig
from transformers.pipelines.pt_utils import KeyDataset
import torch
import numpy as np
import os
import math
from datasets import concatenate_datasets, Dataset
from tqdm import tqdm
import time

In [7]:
# SC_CHANGE
from collections import Counter

In [8]:
# setting env vars
set_seed(1234)
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
import os

os.environ["HF_TOKEN"] = HF_TOKEN

In [10]:
!huggingface-cli login --token $HF_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `Auth` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# Prep Functions

In [11]:
def query_model_w_cs(data, column_to_prompt, num_new_tokens = 200, batch_size=40, num_seeds = 10):
    seeds_list = [1234 + i for i in range(0, num_seeds)]
    all_responses = []
    
    for seed in seeds_list:
        set_seed(seed)
        responses = query_model(data, column_to_prompt, num_new_tokens, batch_size)
        all_responses.append(responses)

    print('...........Acquired responses, 10 samples each.')
    
    most_common_responses = []
    for i in range(len(data)):
        all_i_responses = [response[i] for response in all_responses]
        counter = Counter(all_i_responses)
        most_common_responses.append(counter.most_common(1)[0][0])

    print('...........Found most common responses.')
    
    return most_common_responses

In [12]:
def query_model(data, column_to_prompt, num_new_tokens = 200, batch_size=40):
    print(f"...processing in batches of {batch_size}")
    responses = []
    
    for i in range(0, len(data[column_to_prompt]), batch_size):
        batch = data[column_to_prompt][i:i+batch_size]
        # print(f"...processing batch starting with #{i}. Number of examples: {len(batch)}")

        formatted_input = [tokenizer.apply_chat_template(
                                convo,
                                tokenize=False,
                                add_generation_prompt=True
                            ) for convo in batch
                          ]
        
        model_inputs = tokenizer(formatted_input, padding=True, truncation=True, return_tensors="pt").to(model.device)
    
        with torch.no_grad():
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=num_new_tokens,
            )
        
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        responses += response
    
        # cleaning up memory
        del model_inputs
        del generated_ids
        torch.cuda.empty_cache()

    return responses

In [13]:
def generate_prompts(example, template, instruction):
    prompt = []

    prompt.append({"role": "system",
                  "content": instruction})
    
    for few_shot_example in example['few_shot']:
        few_shot_example_prepped = {}
        few_shot_example_prepped['answer'] = few_shot_example['answers']['text'][0] if len(few_shot_example['answers']['text']) > 0 else 'Cannot be determined'
        few_shot_example_prepped['question'] = few_shot_example['question']
        few_shot_example_prepped['context'] = few_shot_example['context']
        prompt.append({"role": "user",
                      "content": (prompt_template.invoke(few_shot_example_prepped).text)})
        prompt.append({"role": "assistant",
                      "content": few_shot_example_prepped['answer']})
    
    prompt.append({"role": "user",
                  "content": (template.invoke(dict(example)).text)})
     
    example['prompt'] = prompt

    return example

In [14]:
def prompt_for_answer(ds):
    start = time.time()
    output_100 = query_model_w_cs(ds.select(range(100)), "prompt", 50)
    end = time.time()
    print(f"Processing 100 records took {end-start} secs. Estimated completion: {(len(ds) / 100)*(end-start)} secs [{((len(ds) / 100)*(end-start)) / 60} mins]")
    output_next = query_model_w_cs(ds.select(range(100, len(ds))), "prompt", 50)
    print('Done. Combining outputs')
    output = output_100 + output_next
    cleaned_output = [out.strip() for out in output]

    # SC_CHANGE
    if 'predicted_answer_sc' in ds.column_names:
        ds = ds.remove_columns(['predicted_answer_sc'])

    ds = ds.add_column('predicted_answer_sc', cleaned_output)
    return ds

In [15]:
def generate_iter0_answers(dataset, instruction, prompt_template, filedir_prefix_saveto):
    # generating prompts
    dataset['train'] = dataset['train'].map(generate_prompts, 
                                            fn_kwargs={'template':prompt_template,
                                                       'instruction': instruction})
    dataset['train'].to_parquet(f"{filedir_prefix_saveto}prompts.parquet")
    # obtaining answers
    dataset['train'] = prompt_for_answer(dataset['train'])
    
    dataset['train'].to_parquet(f"{filedir_prefix_saveto}results.parquet")

    return dataset

# Load Dataset

In [16]:
from datasets import load_dataset

In [17]:
instruction = "Answer the question based on the provided context. If the answer cannot be determined from the context, say \'Cannot be determined\'."
template = """Context:
{context}

Question:
{question}

Answer:
"""

prompt_template = PromptTemplate.from_template(template)

In [18]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16, 
    bnb_4bit_quant_type="nf4",             
    bnb_4bit_use_double_quant=True,       
)

In [19]:
print(f"LOADING IN MODEL: {MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    # quantization_config=quantization_config,
    torch_dtype=torch.float16,
    cache_dir="/kaggle/working/cache",
    device_map="auto",
    # use_sliding_window=False,
    # max_memory={"cuda:0": "16GB", "cuda:1": "16GB"},
    trust_remote_code=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token_id = 128001
tokenizer.padding_side='left'

LOADING IN MODEL: Qwen/Qwen2.5-1.5B-Instruct-AWQ


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.61G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [20]:
print(f"STARTING PIPELINE")
# generator = pipeline(TASK, model=model, tokenizer = tokenizer)

STARTING PIPELINE


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
print(f"RUNNING ITERATION 0")
dataset = load_dataset("parquet", data_files=DATASET_FILEPATH)

print("- Generating initial responses...")
dataset = generate_iter0_answers(dataset,
                            instruction,
                            prompt_template,
                            FILEDIR)
print('Done.')

RUNNING ITERATION 0


Generating train split: 0 examples [00:00, ? examples/s]

- Generating initial responses...


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/load/serializable.py:289: PydanticDeprecatedSince211: Accessing this attribute on the instance is deprecated, and will be removed in Pydantic V3. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  field = inst.model_fields.get(key)
/usr/local/lib/python3.10/dist-packages/langchain_core/load/serializable.py:213: PydanticDeprecatedSince211: Accessing this attribute on the instance is deprecated, and will be removed in Pydantic V3. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  if k in self.model_fields and self.model_fields[k].exclude:


Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...........Acquired responses, 10 samples each.
...........Found most common responses.
Processing 100 records took 289.22793102264404 secs. Estimated completion: 14461.396551132202 secs [241.02327585220337 mins]
...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...processing in batches of 40
...........Acquired responses, 10 samples each.
...........Found most common responses.
Done. Combining outputs


Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Done.


In [22]:
print("- Adding expected answer")

def generate_expected_answer(example):
    if len(example['answers']['text']) > 0:
        example['expected_answer'] = example['answers']['text'][0]
    else:
        example['expected_answer'] = 'Cannot be determined'

    return example


dataset["train"] = dataset["train"].map(generate_expected_answer)
dataset

- Adding expected answer


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'few_shot', 'prompt', 'predicted_answer_sc', 'expected_answer'],
        num_rows: 5000
    })
})

In [23]:
print("- Adding all answers")

def append_to_all_answers(example):
    example['all_answers'] = [example['predicted_answer_sc']]

    return example


dataset["train"] = dataset["train"].map(append_to_all_answers)
dataset

- Adding all answers


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'few_shot', 'prompt', 'predicted_answer_sc', 'expected_answer', 'all_answers'],
        num_rows: 5000
    })
})

In [24]:
dataset['train'].to_parquet(f"{FILEDIR}results_sc.parquet")

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

28823697